# Parallel computing on multivectors

In [1]:
using Distributed
using DistributedArrays
addprocs(4);

In [2]:
@everywhere using JOLI
@everywhere using LinearAlgebra

#### Setup problem size, define operator, and input

In [3]:
M=8;
NVC=15;

In [4]:
A=joDFT(M,planned=false);
show(A)

Type: joLinearFunction
Name: joDFT
Size: (8, 8)
 DDT: Float64
 RDT: Complex{Float64}


In [5]:
x=randn(M,NVC)

8×15 Array{Float64,2}:
  0.199576   0.432885   0.0204887  …   1.47038    -0.0159484  -0.410357 
  0.845953  -2.00127    0.552835       0.383823   -1.62674     0.0210648
  0.811466   0.485418  -0.760596      -0.211384   -2.16644    -0.72407  
  1.17695   -0.800166  -1.50023       -1.33047    -0.538085   -0.298732 
  0.141245  -0.140401  -0.492261      -0.0149375  -0.808913    0.513472 
 -0.545118   0.300894   0.0444384  …  -1.45843    -0.36587    -0.413716 
  0.921992   0.905117  -0.369768       0.617279   -0.949343    0.720445 
 -0.140138   0.887322   1.25449       -0.678948   -0.270405   -0.461295 

## Using Distributed Ararys

#### Define parallel setup

In [6]:
ps=joPAsetup((M,NVC));
show(ps)

joPAsetup: joPAsetup
 DataType: Float64
 Dims    : (8, 15)
 Chunks  : [1, 4]
 Workers : [2, 3, 4, 5]
  Worker/ranges:   2 (1:8, 1:4)
  Worker/ranges:   3 (1:8, 5:8)
  Worker/ranges:   4 (1:8, 9:12)
  Worker/ranges:   5 (1:8, 13:15)


#### Define parallel operator

In [7]:
pA=joDAdistributedLinOp(A,ps);
show(pA)

Type: joDAdistributedLinearOperator
Name: joDAdistributedLinearOperator(joDFT)
Size: (8, 8)
 NVC: 15
 DDT: Float64
 RDT: Complex{Float64}


#### Define distributed array

In [8]:
@everywhere function fillin(x,n)
    println(myid(),n,(length(n[1]),length(n[2])))
    x[n...]
end

px=dfill(r->fillin(x,r),ps)
px

      From worker 5:	5(1:8, 13:15)(8, 3)
      From worker 3:	3(1:8, 5:8)(8, 4)
      From worker 4:	4(1:8, 9:12)(8, 4)
      From worker 2:	2(1:8, 1:4)(8, 4)


8×15 DArray{Float64,2,Array{Float64,2}}:
  0.199576   0.432885   0.0204887  …   1.47038    -0.0159484  -0.410357 
  0.845953  -2.00127    0.552835       0.383823   -1.62674     0.0210648
  0.811466   0.485418  -0.760596      -0.211384   -2.16644    -0.72407  
  1.17695   -0.800166  -1.50023       -1.33047    -0.538085   -0.298732 
  0.141245  -0.140401  -0.492261      -0.0149375  -0.808913    0.513472 
 -0.545118   0.300894   0.0444384  …  -1.45843    -0.36587    -0.413716 
  0.921992   0.905117  -0.369768       0.617279   -0.949343    0.720445 
 -0.140138   0.887322   1.25449       -0.678948   -0.270405   -0.461295 

### Just parallel

In [9]:
py=pA*px

8×15 DArray{Complex{Float64},2,Array{Complex{Float64},2}}:
     1.2063+0.0im       0.0246775+0.0im       …  -0.372358+0.0im     
  0.0391187-0.637963im  0.0490182+1.1458im       -0.258568+0.361377im
  -0.492371+0.260207im   -0.38822+0.631988im      0.037738-0.129887im
 0.00212725-0.716117im   0.356356+0.849027im     -0.394677-0.660049im
   0.260439+0.0im          1.1654+0.0im           0.442708+0.0im     
 0.00212725+0.716117im   0.356356-0.849027im  …  -0.394677+0.660049im
  -0.492371-0.260207im   -0.38822-0.631988im      0.037738+0.129887im
  0.0391187+0.637963im  0.0490182-1.1458im       -0.258568-0.361377im

### Serial to serial via parallel

#### Define distrtibuting and gathering operators

In [10]:
pD=joDAdistribute(pA);
show(pD)

Type: joDAdistribute
Name: joDAdistributeMV:15
Size: (8, 8)
 NVC: 15
 DDT: Float64
 RDT: Float64


In [11]:
pG=joDAgather(pA);
show(pG)

Type: joDAgather
Name: joDAgatherMV:15
Size: (8, 8)
 NVC: 15
 DDT: Complex{Float64}
 RDT: Complex{Float64}


#### Distribute data, apply operator, gather data

In [12]:
px=pD*x

8×15 DArray{Float64,2,Array{Float64,2}}:
  0.199576   0.432885   0.0204887  …   1.47038    -0.0159484  -0.410357 
  0.845953  -2.00127    0.552835       0.383823   -1.62674     0.0210648
  0.811466   0.485418  -0.760596      -0.211384   -2.16644    -0.72407  
  1.17695   -0.800166  -1.50023       -1.33047    -0.538085   -0.298732 
  0.141245  -0.140401  -0.492261      -0.0149375  -0.808913    0.513472 
 -0.545118   0.300894   0.0444384  …  -1.45843    -0.36587    -0.413716 
  0.921992   0.905117  -0.369768       0.617279   -0.949343    0.720445 
 -0.140138   0.887322   1.25449       -0.678948   -0.270405   -0.461295 

In [13]:
py=pA*px

8×15 DArray{Complex{Float64},2,Array{Complex{Float64},2}}:
     1.2063+0.0im       0.0246775+0.0im       …  -0.372358+0.0im     
  0.0391187-0.637963im  0.0490182+1.1458im       -0.258568+0.361377im
  -0.492371+0.260207im   -0.38822+0.631988im      0.037738-0.129887im
 0.00212725-0.716117im   0.356356+0.849027im     -0.394677-0.660049im
   0.260439+0.0im          1.1654+0.0im           0.442708+0.0im     
 0.00212725+0.716117im   0.356356-0.849027im  …  -0.394677+0.660049im
  -0.492371-0.260207im   -0.38822-0.631988im      0.037738+0.129887im
  0.0391187+0.637963im  0.0490182-1.1458im       -0.258568-0.361377im

In [14]:
y=pG*py

8×15 Array{Complex{Float64},2}:
     1.2063+0.0im       0.0246775+0.0im       …  -0.372358+0.0im     
  0.0391187-0.637963im  0.0490182+1.1458im       -0.258568+0.361377im
  -0.492371+0.260207im   -0.38822+0.631988im      0.037738-0.129887im
 0.00212725-0.716117im   0.356356+0.849027im     -0.394677-0.660049im
   0.260439+0.0im          1.1654+0.0im           0.442708+0.0im     
 0.00212725+0.716117im   0.356356-0.849027im  …  -0.394677+0.660049im
  -0.492371-0.260207im   -0.38822-0.631988im      0.037738+0.129887im
  0.0391187+0.637963im  0.0490182-1.1458im       -0.258568-0.361377im

#### Or all together

In [15]:
ps2s=pG*pA*pD
show(ps2s)

Type: joLinearOperator
Name: ((joDAgatherMV:15*joDAdistributedLinearOperator(joDFT))*joDAdistributeMV:15)
Size: (8, 8)
 DDT: Float64
 RDT: Complex{Float64}


In [16]:
y=ps2s*x

8×15 Array{Complex{Float64},2}:
     1.2063+0.0im       0.0246775+0.0im       …  -0.372358+0.0im     
  0.0391187-0.637963im  0.0490182+1.1458im       -0.258568+0.361377im
  -0.492371+0.260207im   -0.38822+0.631988im      0.037738-0.129887im
 0.00212725-0.716117im   0.356356+0.849027im     -0.394677-0.660049im
   0.260439+0.0im          1.1654+0.0im           0.442708+0.0im     
 0.00212725+0.716117im   0.356356-0.849027im  …  -0.394677+0.660049im
  -0.492371-0.260207im   -0.38822-0.631988im      0.037738+0.129887im
  0.0391187+0.637963im  0.0490182-1.1458im       -0.258568-0.361377im